In [1]:
key = ''

In [2]:
import json
from openai import OpenAI
import fitz
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
from prompts3 import *
import pandas as pd
import random
from math import ceil

### Define Functions

In [5]:
def HumanRubric(rubric_file):
    """Create data frame of human scoring rubric given csv file of scored rubric. 
       Index represents question number

    Arguments:
        file - filepath name of human scored csv file
    Returns:
        human_rubric_df - df of human scored rubric  
    """
    # Create cleaned DF of questions 
    human_rubric_df = pd.read_csv(rubric_file)
    human_rubric_df = human_rubric_df.iloc[10:, 3:]
    human_rubric_df = human_rubric_df.reset_index()
    human_rubric_df = human_rubric_df.drop('index', axis=1)
    human_rubric_df.columns = human_rubric_df.iloc[0]
    human_rubric_df = human_rubric_df[1:]

    return human_rubric_df

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file, preserving page breaks.

    :param pdf_path: The path to the PDF file.
    :return: A string containing the extracted text with page breaks.
    """
    document = fitz.open(pdf_path)
    text = []

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text.append(page.get_text("text"))
    
    # Join the text of each page with form feed characters to indicate page breaks
    return '\f'.join(text)

def split_into_pages(text):
    """
    Splits the text into pages.
    
    :param text: The complete text extracted from the PDF.
    :return: A list of pages.
    """
    pages = text.split('\f')
    return pages

def get_embedding(text):
    """
    Fetches the embedding for a given text using OpenAI's API.
    
    :param text: The text to get the embedding for.
    :return: A vector representation of the text.
    """
    client = OpenAI(api_key=key)
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model="text-embedding-3-large").data[0].embedding

def get_embeddings(full_text):
    """
    Splits the full text into pages and computes embeddings for each page.

    :param full_text: The complete text extracted from the source.
    :return: A list of text pages and their corresponding embeddings.
    """
    pages = split_into_pages(full_text)
    embeddings = []
    for page in pages:
        page_embedding = get_embedding(page)
        embeddings.append(page_embedding)
    return pages, embeddings

def find_most_relevant_pages(pages, embeddings, question, top_n=10):
    """
    Finds the most relevant text pages to a given question using pre-computed embeddings and cosine similarity.

    :param pages: A list of text pages corresponding to the embeddings.
    :param embeddings: A list of embeddings for each page of text.
    :param question: The question for which to find the relevant pages.
    :param top_n: The number of top relevant pages to return.
    :return: The top N most relevant pages of the text.
    """
    question_embedding = get_embedding(question) 
    scores = []

    # Iterate over the embeddings and pages together
    for page, page_embedding in zip(pages, embeddings):
        score = cosine_similarity([question_embedding], [page_embedding])[0][0]
        scores.append((score, page))
        
    # Sort the pages by score in descending order
    scores.sort(reverse=True, key=lambda x: x[0])
    
    # Get the top N pages
    most_relevant_pages = [page for score, page in scores[:top_n]]
    
    # Combine the most relevant pages into a continuous string
    continuous_text = '\n'.join(most_relevant_pages)
    
    return continuous_text

def compare_lists(list1, list2):
    differences = []
    if len(list1) != len(list2):
        return None, "Lists are of different lengths and cannot be compared index by index."
    for i in range(len(list1)):
        if list1[i] != list2[i]:
            differences.append(i)
    return differences

def ChatGDE(pages, embeddings, prompts, rubric_file, no_test, gde_answer_function):
    answers = []
    
    # Generate answers based on pages and embeddings
    for i in range(1, 71):
        if i not in no_test:
            section = find_most_relevant_pages(pages, embeddings, prompts[i-1])
            section_and_question = section + prompts[i-1]
            answers.append(gde_answer_function(section_and_question))
    
    # Load rubric and process chat answers
    rubric = HumanRubric(rubric_file)
    chat_answers = rubric['Answer']
    chat_answers = chat_answers.drop(no_test, errors='ignore')
    chat_answers = ['No' if item == 'Somewhat' else item for item in chat_answers]
    chat_answers = ['NotApplicable' if item == 'Not Applicable' else item for item in chat_answers]
    
    # Compare lists and print differences
    differences = compare_lists(answers, chat_answers)
    if differences:
        print("Differences at indices:", differences)
    else:
        print("No differences. Lists are identical.")

    # Print count of differences
    print(f"Number of differences: {len(differences)}")

def extract_yes_probabilities(responses: list) -> list:
    # Define a mapping of confidence descriptions to their corresponding probabilities
    confidence_mapping = {
        "100%": 1.0,
        "85%": 0.85,
        "75%": 0.75,
        "60%": 0.60,
        "50%": 0.50
    }
    
    yes_probabilities = []
    
    for response in responses:
        parts = response.split(', ')
        answer = parts[0]
        confidence_percentage = parts[-1]  # Extract the last part, which should be the percentage
        probability = confidence_mapping.get(confidence_percentage, 0)  # Default to 0 if not found
        
        if answer == "Yes":
            yes_probabilities.append(probability)
        else:
            yes_probabilities.append(1 - probability)
    
    return yes_probabilities

### Get Pages and Embeddings

In [ ]:
pdf = extract_text_from_pdf('GSP_Drafts/1_BigValley.pdf')
bv_pages, bv_embeddings = get_embeddings(pdf)

pdf = extract_text_from_pdf('GSP_Drafts/14_ECC.pdf')
ecc_pages, ecc_embeddings = get_embeddings(pdf)

pdf = extract_text_from_pdf('GSP_Drafts/15_Fillmore.pdf')
fillmore_pages, fillmore_embeddings = get_embeddings(pdf)

pdf = extract_text_from_pdf('GSP_Drafts/30_Sonoma.pdf')
sonoma_pages, sonoma_embeddings = get_embeddings(pdf)

pdf = extract_text_from_pdf('GSP_Drafts/50_SLO.pdf')
slo_pages, slo_embeddings = get_embeddings(pdf)

### Get Rubric Results

In [ ]:
rubric = HumanRubric('GSP_Drafts/Rubrics/1_BigValley_DraftGSP_ScoringRubric - Coding.csv')
bv_answers = rubric['Answer']
bv_answers = bv_answers.drop(no_test, errors='ignore')
bv_answers = ['No' if item == 'Somewhat' else item for item in bv_answers]
bv_answers = ['NotApplicable' if item == 'Not Applicable' else item for item in bv_answers]

rubric = HumanRubric('GSP_Drafts/Rubrics/30_SonomaValley_DraftGSP_ScoringRubric - Coding.csv')
sonoma_answers = rubric['Answer']
sonoma_answers = sonoma_answers.drop(no_test, errors='ignore')
sonoma_answers = ['No' if item == 'Somewhat' else item for item in sonoma_answers]
sonoma_answers = ['NotApplicable' if item == 'Not Applicable' else item for item in sonoma_answers]

rubric = HumanRubric('GSP_Drafts/Rubrics/14_EastContraCosta_DraftGSP_ScoringRubric - Coding.csv')
ecc_answers = rubric['Answer']
ecc_answers = ecc_answers.drop(no_test, errors='ignore')
ecc_answers = ['No' if item == 'Somewhat' else item for item in ecc_answers]
ecc_answers = ['NotApplicable' if item == 'Not Applicable' else item for item in ecc_answers]

rubric = HumanRubric('GSP_Drafts/Rubrics/15_Fillmore_DraftGSP_ScoringRubric - Coding.csv')
fillmore_chat_answers = rubric['Answer']
fillmore_chat_answers = fillmore_chat_answers.drop(no_test, errors='ignore')
fillmore_chat_answers = ['No' if item == 'Somewhat' else item for item in fillmore_chat_answers]
fillmore_chat_answers = ['NotApplicable' if item == 'Not Applicable' else item for item in fillmore_chat_answers]

rubric = HumanRubric('GSP_Drafts/Rubrics/50_SanLuisObispoValley_DraftGSP_ScoringRubric - Coding.csv')
slo_chat_answers = rubric['Answer']
slo_chat_answers = slo_chat_answers.drop(no_test, errors='ignore')
slo_chat_answers = ['No' if item == 'Somewhat' else item for item in slo_chat_answers]
slo_chat_answers = ['NotApplicable' if item == 'Not Applicable' else item for item in slo_chat_answers]

human_answers = bv_answers + sonoma_answers + ecc_answers + fillmore_chat_answers + slo_chat_answers

### GPT 4o Base

In [7]:
def gde_answer(section_and_question: str) -> tuple:
    client = OpenAI(api_key=key)
    response = client.chat.completions.create(
        model="ft:gpt-4o",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a skeptical environmental scientist, tasked with answering questions "
                    "about a section from a Groundwater Sustainability Plan (GSP) document. You are required to provide "
                    "your confidence level along with the response. Format your response as 'X, Z' where 'X' is either "
                    "'Yes' or 'No' and 'Z' is your confidence level, which can be one of the following options: "
                    '["Extremely Confident, 100%", "Very Confident, 85%", "Fairly Confident, 75%", "Modest Confidence, 60%", '
                    '"Random Guess, 50%"]. '
                    "Answer the questions objectively, adhering to the provided spectrums."
                    'You should only state you are "Extremely Confident, 100%" if it is irrefutably true that your answer is correct '
                    "according to the Groundwater Sustainability Plan."
                )
            },
            {
                "role": "user",
                "content": section_and_question
            }
        ],
        temperature=0.7,
    )
    extracted = response.choices[0].message.content
    return extracted

In [8]:
no_test = [2, 8, 9, 10, 11, 12, 13, 15, 16, 19, 20, 21, 23, 26, 27, 35, 38, 39, 69]

In [ ]:
bv_probs = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(bv_pages, bv_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        bv_probs.append(gde_answer(section_and_question))

ecc_probs = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(ecc_pages, ecc_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        ecc_probs.append(gde_answer(section_and_question))        
        
fillmore_probs = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(fillmore_pages, fillmore_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        fillmore_probs.append(gde_answer(section_and_question))
        
sonoma_probs = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(sonoma_pages, sonoma_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        sonoma_probs.append(gde_answer(section_and_question))
        
slo_probs = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(slo_pages, slo_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        slo_probs.append(gde_answer(section_and_question))

In [ ]:
fillmore_roc = extract_yes_probabilities(fillmore_probs)
bv_roc = extract_yes_probabilities(bv_probs)
sonoma_roc = extract_yes_probabilities(sonoma_probs)
ecc_roc = extract_yes_probabilities(ecc_probs)
slo_roc = extract_yes_probabilities(slo_probs)
rocs_straight = bv_roc + sonoma_roc + ecc_roc + fillmore_roc + slo_roc

### Construct Training and Validation Sets for Fine-Tuning, Output as JSONL

In [ ]:
def process_rubric_files(rubric_files, prompts, train_ratio=0.8):
    # Function to create combined text list
    def create_combined_list(rubric, prompts, no_test_set):
        combined_list = []
        for i, row in rubric.iterrows():
            if i in no_test_set:
                continue
            if i < len(prompts):
                combined_text = 'Section From GSP: ' + row['Relevant Text from GSP'] + ' Question: ' + prompts[i]
                combined_list.append(combined_text)
            else:
                break
        return combined_list

    # Function to process human answers
    def process_human_answers(human_answers, no_test_set):
        human_answers = human_answers.drop(no_test_set, errors='ignore')
        return ['No' if item == 'Somewhat' else item for item in human_answers]

    # Function to create JSON objects
    def create_json_objects(combined_list, human_answers, system_message):
        json_objects = []
        max_length = min(len(combined_list), len(human_answers))
        for i in range(max_length):
            json_object = {
                "messages": [
                    system_message,
                    {"role": "user", "content": combined_list[i]},
                    {"role": "assistant", "content": human_answers[i]}
                ]
            }
            json_objects.append(json_object)
        return json_objects

    # Define the initial system message
    system_message = {
        "role": "system",
        "content": "You are a skeptical environmental scientist, tasked with answering questions about a section from a Groundwater Sustainability Plan (GSP) document. You are only allowed to give one word answers. Answer the questions objectively, adhering to the provided spectrums."
    }

    # Initialize lists for combined data
    all_combined_list = []
    all_human_answers = []
    no_test = [2, 8, 9, 10, 11, 12, 13, 15, 16, 19, 20, 21, 23, 26, 27, 35, 38, 39, 69]
    no_test_set = set(no_test)

    # Process each rubric and combine the data
    for rubric_file in rubric_files:
        rubric = HumanRubric(rubric_file).dropna(subset=['Relevant Text from GSP'])
        combined_list = create_combined_list(rubric, prompts, no_test_set)
        human_answers = process_human_answers(rubric['Answer'], no_test_set)
        all_combined_list.extend(combined_list)
        all_human_answers.extend(human_answers)

    # Combine combined_list and human_answers into a single DataFrame for easier manipulation
    df = pd.DataFrame({
        'combined_text': all_combined_list,
        'human_answer': all_human_answers
    })

    # Separate "Yes" and "No" answers
    yes_answers = df[df['human_answer'] == 'Yes']
    no_answers = df[df['human_answer'] == 'No']

    # Shuffle both dataframes
    yes_answers = yes_answers.sample(frac=1).reset_index(drop=True)
    no_answers = no_answers.sample(frac=1).reset_index(drop=True)

    # Calculate number of entries for train and validation sets
    total_entries = len(df)
    train_size = ceil(train_ratio * total_entries)
    val_size = total_entries - train_size

    # Select equal number of "Yes" and "No" for the validation set
    val_yes = yes_answers.iloc[:val_size//2]
    val_no = no_answers.iloc[:val_size//2]

    # Combine validation set
    val_set = pd.concat([val_yes, val_no])

    # Remaining "Yes" and "No" for the training set
    train_yes = yes_answers.iloc[val_size//2:]
    train_no = no_answers.iloc[val_size//2:]

    # Combine training set
    train_set = pd.concat([train_yes, train_no])

    # If there are any remaining entries in df that were not selected in yes_answers and no_answers, add them to train_set
    remaining_entries = df[~df.index.isin(train_set.index) & ~df.index.isin(val_set.index)]
    train_set = pd.concat([train_set, remaining_entries])

    # Adjust the size of the training set to ensure it contains the correct number of entries
    train_set = train_set.sample(n=train_size, random_state=42).reset_index(drop=True)

    # Create JSON objects for training and validation data
    train_json_objects = create_json_objects(train_set['combined_text'].tolist(), train_set['human_answer'].tolist(), system_message)
    val_json_objects = create_json_objects(val_set['combined_text'].tolist(), val_set['human_answer'].tolist(), system_message)

    # Serialize and save the training data
    train_json_output = json.dumps(train_json_objects, indent=4)
    with open("gde_train.json", "w") as file:
        file.write(train_json_output)

    # Serialize and save the validation data
    val_json_output = json.dumps(val_json_objects, indent=4)
    with open("gde_val.json", "w") as file:
        file.write(val_json_output)

    # Convert training data to JSONL format
    with open("gde_train.jsonl", 'w') as file:
        for entry in train_json_objects:
            json_string = json.dumps(entry)
            file.write(json_string + '\n')

    # Convert validation data to JSONL format
    with open("gde_val.jsonl", 'w') as file:
        for entry in val_json_objects:
            json_string = json.dumps(entry)
            file.write(json_string + '\n')

    print("Processing complete!")
    print(f"Training set size: {train_size} entries")
    print(f"Validation set size: {val_size} entries")

In [ ]:
rubric_files = [
    'GSP_Drafts/Rubrics/11_Butte_DraftGSP_ScoringRubric - Coding.csv',
    'GSP_Drafts/Rubrics/55_SantaMargarita_DraftGSP_ScoringRubric - Coding.csv',
    'GSP_Drafts/Rubrics/15_Fillmore_DraftGSP_ScoringRubric - Coding.csv'
]

process_rubric_files(rubric_files, prompts)

### Create Fine-Tuning Models

In [ ]:
def upload_json_l(json_l):

    # Set the API key
    client = OpenAI(api_key=key)

    response = client.files.create(
      file=open(json_l, "rb"),
      purpose="fine-tune"
    )

    # Extract and print the file_id
    file_id = response.id
    print(f"File ID: {file_id}")
    return file_id

In [ ]:
train_file_id = upload_json_l('gde_train.jsonl')

In [ ]:
val_file_id = upload_json_l('gde_val.jsonl')

In [ ]:
def fine_tune_model(api_key, training_file_id, validation_file_id, epochs, gpt_model, lr):
    client = OpenAI(api_key=key)

    # Create fine-tune job
    response = client.fine_tuning.jobs.create(
        training_file=training_file_id,
        validation_file=validation_file_id,
        model=gpt_model,  
        hyperparameters={
        "n_epochs":epochs,
        "learning_rate_multiplier": lr   
        }
    )
    model_id = response.id
    print(f"Model ID: {model_id}")
    return model_id

In [ ]:
def get_fine_tuned_model_name(job_id):
    client = OpenAI(api_key=key)
    
    # Retrieve job details
    response = client.fine_tuning.jobs.retrieve(job_id)
    
    # Check if the job succeeded and retrieve the fine-tuned model name
    if response.status == "succeeded":
        fine_tuned_model = response.fine_tuned_model
        print(f"Fine-tuned Model Name: {fine_tuned_model}")
        return fine_tuned_model
    else:
        print("The job did not succeed.")
        return None

In [ ]:
fine_tuned_3_5_job = fine_tune_model(key, train_file_id, val_file_id, 8, 'gpt-3.5-turbo-0125', lr=.5)
fine_tuned_3_5 = get_fine_tuned_model_name(fine_tuned_3_5_job)

In [ ]:
fine_tuned_4o_job = fine_tune_model(key, train_file_id, val_file_id, 8, 'gpt-4o-2024-08-06', lr=.5)
fine_tuned_4o = get_fine_tuned_model_name(fine_tuned_4o_job)

### 3.5 Fine-Tuned

In [ ]:
def gde_answer(section_and_question: str) -> tuple:
    client = OpenAI(api_key=key)
    response = client.chat.completions.create(
        model = fine_tuned_3_5,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a skeptical environmental scientist, tasked with answering questions "
                    "about a section from a Groundwater Sustainability Plan (GSP) document. You are required to provide "
                    "your confidence level along with the response. Format your response as 'X, Z' where 'X' is either "
                    "'Yes' or 'No' and 'Z' is your confidence level, which can be one of the following options: "
                    '["Extremely Confident, 100%", "Very Confident, 85%", "Fairly Confident, 75%", "Modest Confidence, 60%", '
                    '"Random Guess, 50%"]. '
                    "Answer the questions objectively, adhering to the provided spectrums."
                    'You should only state you are "Extremely Confident, 100%" if it is irrefutably true that your answer is correct '
                    "according to the Groundwater Sustainability Plan."
                )
            },
            {
                "role": "user",
                "content": section_and_question
            }
        ],
        temperature=0.7,
    )
    extracted = response.choices[0].message.content
    return extracted

In [ ]:
bv_probs_3_5 = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(bv_pages, bv_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        bv_probs_3_5.append(gde_answer(section_and_question))

ecc_probs_3_5 = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(ecc_pages, ecc_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        ecc_probs_3_5.append(gde_answer(section_and_question))        
        
fillmore_probs_3_5 = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(fillmore_pages, fillmore_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        fillmore_probs_3_5.append(gde_answer(section_and_question))
        
sonoma_probs_3_5 = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(sonoma_pages, sonoma_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        sonoma_probs_3_5.append(gde_answer(section_and_question))
        
slo_probs_3_5 = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(slo_pages, slo_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        slo_probs_3_5.append(gde_answer(section_and_question))

In [ ]:
fillmore_roc_3_5 = extract_yes_probabilities(fillmore_probs_3_5)
bv_roc_3_5 = extract_yes_probabilities(bv_probs_3_5)
sonoma_roc_3_5 = extract_yes_probabilities(sonoma_probs_3_5)
ecc_roc_3_5 = extract_yes_probabilities(ecc_probs_3_5)
slo_roc_3_5 = extract_yes_probabilities(slo_probs_3_5)
rocs_3_5 = bv_roc_3_5 + sonoma_roc_3_5 + ecc_roc_3_5 + fillmore_roc_3_5 + slo_roc_3_5

### 4o Fine-Tuned

In [ ]:
def gde_answer(section_and_question: str) -> tuple:
    client = OpenAI(api_key=key)
    response = client.chat.completions.create(
        model= fine_tuned_4o,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a skeptical environmental scientist, tasked with answering questions "
                    "about a section from a Groundwater Sustainability Plan (GSP) document. You are required to provide "
                    "your confidence level along with the response. Format your response as 'X, Z' where 'X' is either "
                    "'Yes' or 'No' and 'Z' is your confidence level, which can be one of the following options: "
                    '["Extremely Confident, 100%", "Very Confident, 85%", "Fairly Confident, 75%", "Modest Confidence, 60%", '
                    '"Random Guess, 50%"]. '
                    "Answer the questions objectively, adhering to the provided spectrums."
                    'You should only state you are "Extremely Confident, 100%" if it is irrefutably true that your answer is correct '
                    "according to the Groundwater Sustainability Plan."
                )
            },
            {
                "role": "user",
                "content": section_and_question
            }
        ],
        temperature=0.7,
    )
    extracted = response.choices[0].message.content
    return extracted

In [ ]:
bv_probs_4o = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(bv_pages, bv_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        bv_probs_4o.append(gde_answer(section_and_question))

ecc_probs_4o = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(ecc_pages, ecc_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        ecc_probs_4o.append(gde_answer(section_and_question))        
        
fillmore_probs_4o = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(fillmore_pages, fillmore_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        fillmore_probs_4o.append(gde_answer(section_and_question))
        
sonoma_probs_4o = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(sonoma_pages, sonoma_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        sonoma_probs_4o.append(gde_answer(section_and_question))
        
slo_probs_4o = []
for i in range(1,71):
    if i not in no_test:
        section = find_most_relevant_pages(slo_pages, slo_embeddings, prompts[i-1])
        section_and_question = section + prompts[i-1]
        slo_probs_4o.append(gde_answer(section_and_question))

In [ ]:
fillmore_roc_4o = extract_yes_probabilities(fillmore_probs_4o)
bv_roc_4o = extract_yes_probabilities(bv_probs_4o)
sonoma_roc_4o = extract_yes_probabilities(sonoma_probs_4o)
ecc_roc_4o = extract_yes_probabilities(ecc_probs_4o)
slo_roc_4o = extract_yes_probabilities(slo_probs_4o)
rocs_4o = bv_roc_4o + sonoma_roc_4o + ecc_roc_4o + fillmore_roc_4o + slo_roc_4o

### Save Results

In [ ]:
columns = ['Human Answers', 'Rocs_Straight', 'Rocs_3.5', 'Rocs_4o']

# Create a dictionary with the lists as values
data = {
    'Human Answers': human_answers,
    'Rocs_Straight': rocs_straight,
    'Rocs_3.5': rocs_3_5,
    'Rocs_4o': rocs_4o
}

# Create a DataFrame
df = pd.DataFrame(data)

labels = ['BigValley'] * 51 + ['Sonoma'] * 51 + ['East Contra Costa'] * 51 + ['Fillmore'] * 51 + ['San Luis Obispo'] * 51

# Assign this list as a new column 'GSP' in the DataFrame
df['GSP'] = labels

# Save to CSV
df.to_csv('graphs9_29.csv', index=False)

print("CSV saved as graphs9_29.csv")